In [1]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [2]:
os.listdir()

['.ipynb_checkpoints',
 'austin_extact_code.ipynb',
 'boston_extract_code.ipynb',
 'chattanooga_extract_code.ipynb',
 'chicago_extract_code.ipynb',
 'creds',
 'kc_extract_code.ipynb',
 'littlerock_extract_code.ipynb',
 'nyc_extract_code.ipynb',
 'SF_extract_code.ipynb']

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



gc = gspread.authorize(credentials)

wks = gc.open("NYTimes Mini Crossword times").sheet1


data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)


df = df.replace(np.NaN, -1)

# saving a data frame to a buffer (same as with a regular file):
df.to_csv('nytimescrossword.csv',index=False)

# NYC

#Initial Load
nyc_table_id = "opendatadbt.311.nyc311"
nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=10000&$where=created_date >= '2021-03-18T00:00:00.000' OR closed_date >= '2021-03-18T00:00:00.000'" )
nyc311json = nyc311.json()
nyc_df = pd.DataFrame(nyc311json)
#nyc_df = nyc_df[['created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']]

In [4]:
#Incremental load
nyc_table_id = "opendatadbt.311.nyc311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.nyc311`
limit 10
"""

nycmaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxopendatedate = str(nycmaxopendatedataframe['f0_'][0])

nycmaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
nycmaxclosedate = str(nycmaxclosedatedataframe['f1_'][0])


In [5]:
nycmaxopendatedate

'2021-03-26T00:59:21.000'

In [6]:

nyc311 = requests.get("https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=50000&$where=created_date>'"+nycmaxopendatedate+"' OR closed_date between '" +nycmaxclosedate+ "' AND '" + current_datetime + "'") 
nyc311json = nyc311.json()
cols = ['unique_key','created_date','closed_date','agency','agency_name','descriptor','incident_zip','incident_address','city','status','latitude','longitude']
nyc_df = pd.DataFrame(nyc311json, columns=cols)
nyc_df

,unique_key,created_date,closed_date,agency,agency_name,descriptor,incident_zip,incident_address,city,status,latitude,longitude
0,49883223,2021-02-27T17:37:40.000,2021-03-26T12:31:04.000,HPD,Department of Housing Preservation and Develop...,PESTS,10029,237 EAST 115 STREET,NEW YORK,Closed,40.796620057618014,-73.93900222373162
1,49760116,2021-02-10T15:36:52.000,2021-03-26T20:30:52.000,HPD,Department of Housing Preservation and Develop...,BATHTUB/SHOWER,10024,103 WEST 80 STREET,NEW YORK,Closed,40.78276991011247,-73.97529037066195
2,49757012,2021-02-10T10:42:57.000,2021-03-26T15:51:57.000,HPD,Department of Housing Preservation and Develop...,BELL/BUZZER/INTERCOM,10472,1742 EAST 172 STREET,BRONX,Closed,40.83258373587881,-73.86920068822464
3,49757334,2021-02-10T10:42:57.000,2021-03-26T15:52:02.000,HPD,Department of Housing Preservation and Develop...,PESTS,10472,1742 EAST 172 STREET,BRONX,Closed,40.83258373587881,-73.86920068822464
4,49950094,2021-03-06T11:58:14.000,2021-03-26T12:25:26.000,DOT,Department of Transportation,Blocked - Construction,10024,347 WEST 87 STREET,NEW YORK,Closed,40.79007155277428,-73.97855230415891
...,...,...,...,...,...,...,...,...,...,...,...,...
8558,47846732,2020-10-11T19:11:52.000,2021-03-26T13:57:25.000,DPR,Department of Parks and Recreation,For One Address,10019,360 WEST 51 STREET,NEW YORK,Closed,40.763800279458756,-73.98777324607715
8559,47847716,2020-10-11T18:50:42.000,2021-03-26T14:12:29.000,DPR,Department of Parks and Recreation,For One Address,10019,252 WEST 51 STREET,NEW YORK,Closed,40.762468700395154,-73.9846112754587
8560,46330584,2020-05-29T09:31:38.000,2021-03-26T15:36:49.000,DPR,Department of Parks and Recreation,Planted More Than 2 Years Ago,NaN,NaN,NaN,Closed,NaN,NaN
8561,45904654,2020-03-27T10:36:12.000,2021-03-26T19:59:40.000,HPD,Department of Housing Preservation and Develop...,REFRIGERATOR,11230,1305 EAST 18 STREET,BROOKLYN,Closed,40.619036650261194,-73.95730817509532


In [7]:
nyc_table_id = "opendatadbt.311.nyc311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('nyc311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    nyc_df, table_ref, job_config=job_config
).result()  # Make an API request.
